# Movie Reviews Sentiment Analysis

### Problem Statement
- In this project, we try to estimate the sentiment from a movie review.

### Dataset
- Source - https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
- The data contains 50,000 reviews, 25,000 of them are positive and 25,000 of them are negative.

In this notebook, we perform training on following models - 
- Logistic Regresssion

### Importing required packages

In [37]:
import numpy as np 
import scipy
import pandas as pd
import re
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from nltk.stem.porter import PorterStemmer
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline  
from sklearn.linear_model import LogisticRegression 
import pickle
import os

### Importing data

In [2]:
df = pd.read_csv('data/imdb_data.csv')
df.head()

,Review,Rating,Sentiment
0,Imagine The Big Chill with a cast of twenty-so...,2,0
1,I'd have to say that I've seen worse Sci Fi Ch...,3,0
2,Director Fabio Barreto got a strange Academy N...,1,0
3,Pretty bad PRC cheapie which I rarely bother t...,4,0
4,This is a very intriguing short movie by David...,8,1


In [4]:
print(f'Shape = {df.shape}')

Shape = (50000, 3)


### Preprocessing text

In [6]:
def text_preprocessing(text):
    # Replacing n't with not since it could be really important in sentiment analysis
    text = re.sub("n't", ' not ', text)
    # Removing URLs
    text = re.sub('(http).*\/', ' ', text)
    # Removing HTML tags
    text = re.sub('<[^>]*>', '', text)
    # Extracting emoticons
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|P|D|]|})', text)
    # Removing punctuations
    text = re.sub('[\W]+', ' ', text.lower())
    # Adding emoticons at end and converting :-) to :)
    text = text + ' ' + ' '.join(emoticons).replace('-', '')
    return text   

In [7]:
df['Review'] = df['Review'].apply(text_preprocessing)
df.head()

,Review,Rating,Sentiment
0,imagine the big chill with a cast of twenty so...,2,0
1,i d have to say that i ve seen worse sci fi ch...,3,0
2,director fabio barreto got a strange academy n...,1,0
3,pretty bad prc cheapie which i rarely bother t...,4,0
4,this is a very intriguing short movie by david...,8,1


### Creating Feature Matrix and Target Vector

In [8]:
X = df['Review'].values 
y = df['Sentiment'].values 

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42, stratify=y)

Tokenizer-1 : we simply split the text 

In [13]:
def tokenizer(text):
    return text.split()

Tokenizer-2 : we use a tokenizer which contains stems of words as tokens. We use PorterStemmer() module to stem the words.

In [17]:
porter = PorterStemmer()
def tokenizer_porter(text):
    return [porter.stem(word) for word in text.split()]

Removing Stopwords

In [21]:
nltk.download('stopwords')
from nltk.corpus import stopwords
stop = stopwords.words('english')


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\mndpp\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [22]:
print(f'Number of stopwords = {len(stop)}')

Number of stopwords = 179


In [23]:
print(f'Stopwords = {stop}')

Stopwords = ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so

There are certain stopwords which are relevant so we don't to remove those words

In [26]:
relevant_stopwords = ['no', 'not', 'ain', 'don', "don't", 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn', "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'mightn', "mightn't", 'mustn', "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", 'won', "won't", 'wouldn', "wouldn't"]
new_stop = list(set(stop) - set(relevant_stopwords))
print(f'Stopwords remaining = {len(new_stop)}')

Stopwords remaining = 140


### TfidfVectorizer

In [28]:
tfidf = TfidfVectorizer(lowercase = False,
                        preprocessor=None,
                        min_df=5)

### Pipeline

In [30]:
lr_tfidf_pipe = Pipeline([
    ('vect', tfidf),
    ('clf', LogisticRegression(solver='liblinear'))
])

In [33]:
np.random.seed(42)
param_range = scipy.stats.loguniform(0.0001, 1000.0)
param_grid_rs = [
    {
        'vect__ngram_range' : [(1, 1), (1, 2)],
        'vect__stop_words'  : [None, new_stop],
        'vect__tokenizer'   : [tokenizer, tokenizer_porter],
        'clf__penalty'      : ['l1', 'l2'],
        'clf__C'            : param_range
    },
    {
        'vect__ngram_range' : [(1, 1), (1, 2)],
        'vect__stop_words'  : [None, new_stop],
        'vect__tokenizer'   : [tokenizer, tokenizer_porter],
        'vect__use_idf'     : [False],
        'vect__norm'        : [None],
        'clf__penalty'      : ['l1','l2'],
        'clf__C'            : param_range
    }
]
rs_lr = RandomizedSearchCV(lr_tfidf_pipe, param_grid_rs,
                           scoring='accuracy', cv=5,
                           verbose = 2, n_jobs= -1,
                           random_state = 42, n_iter = 100)
rs_lr.fit(X_train, y_train)

Fitting 5 folds for each of 100 candidates, totalling 500 fits


c:\Users\mndpp\Desktop\Github_Projects\01_movie_review_sentiment\venv\lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


RandomizedSearchCV(cv=5,
                   estimator=Pipeline(steps=[('vect',
                                              TfidfVectorizer(lowercase=False,
                                                              min_df=5)),
                                             ('clf',
                                              LogisticRegression(solver='liblinear'))]),
                   n_iter=100, n_jobs=-1,
                   param_distributions=[{'clf__C': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x0000021FAD2FBB50>,
                                         'clf__penalty': ['l1', 'l2'],
                                         'vect__ngram_range': [(1, 1), (1, 2)],
                                         'vec...
                                                               'how', 'him',
                                                               'only', 'very',
                                                               'ours', 'if',
                                                               'both', 'of',
                                                               'will', 'down',
                                                               'can', 'who',
                                                               'for', 'same',
                                                               'y', 'me', 'by',
                                                               'am', 'herself',
                                                               'an', 'off',
                                                               'or', 'here',
                                                               'his', "you've",
                                                               'most', 'some',
                                                               'when', ...]],
                                         'vect__tokenizer': [<function tokenizer at 0x0000021F8B9D0280>,
                                                             <function tokenizer_porter at 0x0000021F8B8375B0>],
                                         'vect__use_idf': [False]}],
                   random_state=42, scoring='accuracy', verbose=2)

### Post Training

In [34]:
print(f'Accuracy of best model : {rs_lr.best_score_:.3f}')
print(f'Best Parameters = {rs_lr.best_params_}')
best_classifier = rs_lr.best_estimator_
best_classifier.fit(X_train, y_train)
print(f'Accuracy on test set = {best_classifier.score(X_test, y_test)}')

Accuracy of best model : 0.903
Best Parameters = {'clf__C': 0.05809926352311592, 'clf__penalty': 'l2', 'vect__ngram_range': (1, 2), 'vect__norm': None, 'vect__stop_words': None, 'vect__tokenizer': <function tokenizer at 0x0000021F8B9D0280>, 'vect__use_idf': False}


c:\Users\mndpp\Desktop\Github_Projects\01_movie_review_sentiment\venv\lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Accuracy on test set = 0.90188


### Saving model

In [38]:
if not os.path.exists('models'):
    os.mkdir('models')
model_filename = 'models/lr_model.pickle'
pickle.dump(best_classifier, open(model_filename, 'wb'))